In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
!pip install -q transformers accelerate langchain_huggingface bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.11 requires langchain-core<0.2,>=0.1.29, but you have langchain-core 0.2.19 which is incompatible.
langchain-community 0.0.27 requires langchain-core<0.2.0,>=0.1.30, but you have langchain-core 0.2.19 which is incompatible.
langchain-text-splitters 0.0.1 requires langchain-core<0.2.0,>=0.1.28, but you have langchain-core 0.2.19 which is incompatible.
langchain-openai 0.1.15 requires openai<2.0.0,>=1.32.0, but you have openai 1.13.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip -q install torch==2.1.0

ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1)
ERROR: No matching distribution found for torch==2.1.0

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from langchain_huggingface import HuggingFacePipeline
from transformers import BitsAndBytesConfig, pipeline
from datetime import datetime
from functools import cache

C:\Users\Vu Diep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if torch.cuda.is_available():
    print("CUDA is available! PyTorch can use GPU.")
else:
    print("CUDA is not available. PyTorch will use CPU.")

CUDA is not available. PyTorch will use CPU.


In [8]:
@cache
def get_tokenizer_model():
  model = AutoModelForCausalLM.from_pretrained(
      "meta-llama/Llama-2-7b-chat-hf",
      device_map="auto",
      # quantization_config=BitsAndBytesConfig(load_in_8bit=True),
      cache_dir="./cache"
  )
  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='./cache')
  return model, tokenizer

In [ ]:
model, tokenizer = get_tokenizer_model()

In [12]:
def generate_output(input_text):
  inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
  output = model.generate(
      **inputs,
      streamer=streamer,
      max_new_tokens=float("inf"),
      use_cache=True
  )
  output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return output_text

## Flask API for llama-2 model



In [ ]:
!pip -q install fastapi uvicorn pyngrok colabcode nest-asyncio

In [10]:
!ngrok config add-authtoken 2IeOitDBgJ6jr7xKap3DQAzkaqs_2SAicHBCB7vJAwB6Johxr

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
from colabcode import ColabCode
from pyngrok import ngrok
import nest_asyncio
import uvicorn

app = FastAPI()

class TextItem(BaseModel):
    text: str

@app.post("/post")
def create_item(item: TextItem):
    output = generate_output(item.text)
    return {"content": output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://a5bc-34-16-181-235.ngrok-free.app


INFO:     Started server process [413]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     45.51.26.112:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     45.51.26.112:0 - "GET /openapi.json HTTP/1.1" 200 OK


I'm doing well, thanks for asking! How about you?

It's great to hear from you, and I hope you're having a wonderful day. Is there anything new and exciting happening in your life?

I'm looking forward to catching up with you and hearing all about what's been going on. Let me know if you have any news or updates to share.

Take care and talk to you soon!
INFO:     45.51.26.112:0 - "POST /post HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [413]


KeyboardInterrupt: 